## Analýza článků v časopisu Guardian

V tomto notebooku se budeme věnovat zpracování nestrukturovaných dat, které jsou náhodně zvolenou podmnožinou [většího datasetu z Kaggle](https://www.kaggle.com/datasets/adityakharosekar2/guardian-news-articles?resource=download).

Nejprve nastavíme klienta pro OpenAI API. Toto nastavení se bude lišit v závislosti na tom, zda používáte Google Colab nebo spouštíte notebook na vlastním počítači, ale pokud následující dvě buňky proběhnou bez chyby, jste připraveni pokračovat.

In [4]:
try:
    from google.colab import userdata
    _secret = userdata.get("OPENAI_API_KEY")
    %pip install pandas instructor openai python-dotenv rich
except ImportError:
    import os
    from dotenv import load_dotenv
    _ = load_dotenv()
    _secret = os.environ.get("OPENAI_API_KEY")

In [5]:
import instructor
from openai import OpenAI
import os
from rich import print

try:
    if not _secret:
        raise ValueError("API klíč nebyl nastaven!")
except NameError:
    print("Nastavte si API klíč v proměnné GOOGLE_API_KEY a znovu spusťte *celý* notebook včetně předchozí buňky")

_client = OpenAI()
client = instructor.from_openai(_client)
test = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, how are you? Respond with an emotion."},
    ],
    model="gpt-4o",
    response_model=str,
)
print(test)


/Users/simon/Documents/czechitas-ai-data/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


I'm feeling curious and ready to help you with any questions or tasks you have! How can I assist you today?

## Čtení dat

In [7]:
import pandas as pd

data = pd.read_csv('guardian_100.csv')
print(data.shape)
data.head()


(100, 7)

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,books/2022/may/24/margaret-atwood-handmaids-ta...,Books,Atwood responds to book bans with ‘unburnable’...,https://www.theguardian.com/books/2022/may/24/...,Amid political firestorms over books deemed by...,2022-05-24 14:52:56,147533
1,uk-news/2022/apr/29/eden-project-cornwall-inst...,UK news,Eden Project installs plastic grass to stop ch...,https://www.theguardian.com/uk-news/2022/apr/2...,The Eden Project in Cornwall has installed pla...,2022-04-29 09:24:02,144415
2,music/2018/jan/26/grammys-2018-jay-z-despacito...,Music,Who will – and should – win the major categori...,https://www.theguardian.com/music/2018/jan/26/...,"Album of the year Awaken, My Love! – Childish ...",2018-01-26 12:11:12,47554
3,technology/2018/apr/24/facebook-replacement-op...,Technology,Is Facebook replaceable? Tech investor launche...,https://www.theguardian.com/technology/2018/ap...,Can Facebook be replaced? The prominent Silico...,2018-04-24 19:35:23,53471
4,us-news/2019/jun/30/democrats-impeach-trump-pe...,US news,Democrats divided as pressure to impeach build...,https://www.theguardian.com/us-news/2019/jun/3...,In the House of Representatives the apparently...,2019-06-30 04:00:06,79238


## Úkoly

1. Vytvořte a implementujte prompt, který shrne každý z článků do jedné věty.
2. Vytvořte prompt, který z textu článku vytvoří strukturovaný výstup, ve kterém bude uvedeno, jakých lidí/entit se týká, do jaké z předdefinovaných kategorií (`data['sectionName'].unique().tolist()`) spadá, a jak urgentní popisuje situaci. Uložte tuto strukturu do nového sloupce `data['structured_output']`.
3. Udělejte graf častosti výskytu jednotlivých kategorií a jednotlivých entit. (Fantazii se meze nekladou - prodiskutujte to s pomocníčkem.)
4. Porovnejte výsledky klasifikace s původním sloupcem `data['sectionName']`. Jak velké úspěšnosti jste dosáhly?

## Ukázka

In [8]:
example = data[0:2]
outputs = []
for i, row in example.iterrows():
    print(row['webTitle'])
    output = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes news articles down to exactly one sentence."},
            {"role": "user", "content": row['bodyContent']},
        ],
        model="gpt-4o",
        response_model=str,
    )
    print(output)
    outputs.append(output)


Atwood responds to book bans with ‘unburnable’ edition of Handmaid’s Tale

Margaret Atwood has announced an "unburnable" edition of her novel The Handmaid's Tale, made with fireproof 
materials to symbolize against censorship, with proceeds from its Sotheby's auction supporting Pen America's free 
expression work amid ongoing debates about book bans in schools.

Eden Project installs plastic grass to stop children getting muddy

The use of artificial grass in the UK, highlighted by the Eden Project's installation at a children's play area, is
stirring controversy among environmental campaigners due to ecological concerns and criticisms of greenwashing, 
despite claims of durability and air-purifying properties by manufacturers.